In [1]:
conda install pytorch torchvision cudatoolkit=10.2 -c pytorch

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers

In [3]:
pip install pytorch-pretrained-bert==0.4.0

Note: you may need to restart the kernel to use updated packages.


In [4]:
'''
LOADING THE DATASET
'''
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

import pandas as pd
import io
import re

data = pd.read_csv('C:\\Users\in608\BERT_SignTagger/annotatedSignSequences.csv', sep='\t', encoding='utf8', engine='python').fillna(method='ffill')
data.tail(30)

,Unnamed: 0,POSITION,TOKEN,POS,IsAsign,TAG
344259,344259,15350_59,.,.,M:N,ESMN
344260,344260,15351_1,That,DT,M:Y,SPECIAL
344261,344261,15351_2,is,VBZ,M:N,SPECIAL
344262,344262,15351_3,to,TO,M:N,SPECIAL
344263,344263,15351_4,say,VB,M:N,SPECIAL
344264,344264,15351_5,",",",",M:Y,SSCCV
344265,344265,15351_6,he,PRP,M:N,SSCCV
344266,344266,15351_7,felt,VBD,M:N,SSCCV
344267,344267,15351_8,like,IN,M:N,SSCCV
344268,344268,15351_9,a,DT,M:N,SSCCV


In [30]:
import torch

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from torch.optim import Adam

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__

'1.5.1'

In [6]:
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertForSequenceClassification, BertAdam
# from transformers import BertTokenizer
# from transformers import BertForTokenClassification, BertForSequenceClassification, AdamW

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [7]:
# First replacing all nan values in the TAG column of data with NA
data["TAG"].fillna("NA", inplace = True)

print('#####>') 
print(data.tail(10))



# data.columns = ['POSITION', 'TOKEN', 'POS' 'IsAsign', 'TAG']

position_strings = data['POSITION'].values.tolist()


'''
Stripping square brackets off signs because they are not handled well by BERT
'''
data['TOKEN'].replace({'\[([^\]]+)\]' : '\\1'}, inplace=True, regex=True)

'''
MY DATASET DOESN'T HAVE A COLUMN FOR SENTENCE NUMBER. INSTEAD IT HAS A COLUMN 
PROVIDING SENTENCE AND WORD POSITION AS A SINGLE STRING. HERE, I IDENTIFY THE 
SENTENCE NUMBERS AND ADD THEM INTO A NEW COLUMN OF THE DATAFRAME.
'''
print(len(position_strings))
print(len(data))

sentence_numbers = []
for ps in position_strings:
  ups = re.match('([0-9]+)\_', ps).group(1)
  sentence_numbers.append(ups)

print(data.tail(10))


data['SENTENCE_NUMBER'] = sentence_numbers

data.head(30)

#####>
        Unnamed: 0  POSITION      TOKEN  POS IsAsign    TAG
344279      344279   15352_1      Henry  NNP     M:N  SSCCV
344280      344280   15352_2          ,    ,     M:Y   SSMV
344281      344281   15352_3       awed   JJ     M:N   SSMV
344282      344282   15352_4         by   IN     M:N   SSMV
344283      344283   15352_5  authority   NN     M:N   SSMV
344284      344284   15352_6          ,    ,     M:Y   ESMV
344285      344285   15352_7   relapsed   JJ     M:N   ESMV
344286      344286   15352_8       into   IN     M:N   ESMV
344287      344287   15352_9    silence   NN     M:N   ESMV
344288      344288  15352_10          .    .     M:N   ESMV
344289
344289
        Unnamed: 0  POSITION      TOKEN  POS IsAsign    TAG
344279      344279   15352_1      Henry  NNP     M:N  SSCCV
344280      344280   15352_2          ,    ,     M:Y   SSMV
344281      344281   15352_3       awed   JJ     M:N   SSMV
344282      344282   15352_4         by   IN     M:N   SSMV
344283      344283 

,Unnamed: 0,POSITION,TOKEN,POS,IsAsign,TAG,SENTENCE_NUMBER
0,0,1_1,One,CD,M:N,NA,1
1,1,1_2,note,NN,M:N,NA,1
2,2,1_3,ended,VBD,M:N,NA,1
3,3,1_4,with,IN,M:N,NA,1
4,4,1_5,the,DT,M:N,NA,1
5,5,1_6,words,NNS,M:N,NA,1
6,6,1_7,:,:,M:Y,SSCM,1
7,7,1_8,',POS,M:N,SSCM,1
8,8,1_9,School,NNP,M:N,SSCM,1
9,9,1_10,is,VBZ,M:N,SSCM,1


In [8]:
'''
But here, we only want to get those sentences that are in training data, so we 
want to reduce data to the training portion

Selecting portions of the input csv file to be used for training and testing
'''
print(data.tail(10))
print('------>', len(data))

data_type  = []
x=0
while x < 344289:
  if x < 344289:
    data_type.append('TRAINING')
  x += 1



#print('len(data):', len(data), len(data_type))
data['TYPE'] = data_type


#print('data_type:', data_type)
print('data:\n', data)

        Unnamed: 0  POSITION      TOKEN  POS IsAsign    TAG SENTENCE_NUMBER
344279      344279   15352_1      Henry  NNP     M:N  SSCCV           15352
344280      344280   15352_2          ,    ,     M:Y   SSMV           15352
344281      344281   15352_3       awed   JJ     M:N   SSMV           15352
344282      344282   15352_4         by   IN     M:N   SSMV           15352
344283      344283   15352_5  authority   NN     M:N   SSMV           15352
344284      344284   15352_6          ,    ,     M:Y   ESMV           15352
344285      344285   15352_7   relapsed   JJ     M:N   ESMV           15352
344286      344286   15352_8       into   IN     M:N   ESMV           15352
344287      344287   15352_9    silence   NN     M:N   ESMV           15352
344288      344288  15352_10          .    .     M:N   ESMV           15352
------> 344289
data:
         Unnamed: 0  POSITION     TOKEN  POS IsAsign   TAG SENTENCE_NUMBER  \
0                0       1_1       One   CD     M:N    NA        

In [9]:
'''
A FUNCTION THAT CREATES SEQUENCES OF TOKENS IN THE DATAFRAME THAT SHARE THE SAME
SENTENCE_NUMBER
'''
class SentenceGetter(object):
  
  def __init__(self, data):
    self.n_sent = 1
    self.data = data
    self.empty = False
    agg_func = lambda s: [(w, p, t) for w, p, t in zip(s['TOKEN'].values.tolist(), 
                                                       s['POS'].values.tolist(), 
                                                       s['TAG'].values.tolist())]
    #    self.grouped = self.data.groupby(['SENTENCE_NUMBER']).apply(agg_func)
    '''
Change made below so that sentences are sorted in the following order:
HEALTH_TESTING [0:1003], 
LITERATURE_TESTING [1004:1630], 
NEWS_TESTING [1631:2633], 
TRAINING (INCLUDING DAILY MAIL) [2634:]
TRAINING (EXCLUDING DAILY MAIL) [2634:5308]
    '''
    self.grouped = self.data.groupby(['TYPE', 'SENTENCE_NUMBER']).apply(agg_func)
#    self.grouped = self.data.groupby(['SENTENCE_NUMBER']).apply(agg_func)
    self.sentences = [s for s in self.grouped]
#    print('+++>', self.grouped)

  def get_next(self):
    try:
      s = self.grouped['SENTENCE: {}'.format(self.n_sent)]
      self.n_sent += 1
      return s
    except:
      return None

In [10]:
print(data.head(50))
getter = SentenceGetter(data)

    Unnamed: 0 POSITION       TOKEN   POS IsAsign    TAG SENTENCE_NUMBER  \
0            0      1_1         One    CD     M:N     NA               1   
1            1      1_2        note    NN     M:N     NA               1   
2            2      1_3       ended   VBD     M:N     NA               1   
3            3      1_4        with    IN     M:N     NA               1   
4            4      1_5         the    DT     M:N     NA               1   
5            5      1_6       words   NNS     M:N     NA               1   
6            6      1_7           :     :     M:Y   SSCM               1   
7            7      1_8           '   POS     M:N   SSCM               1   
8            8      1_9      School   NNP     M:N   SSCM               1   
9            9     1_10          is   VBZ     M:N   SSCM               1   
10          10     1_11          an    DT     M:N   SSCM               1   
11          11     1_12    obstacle    NN     M:N   SSCM               1   
12          

In [11]:
'''
LOADING ONE OF BERT'S PRETRAINED TOKENIZERS AND DEFINED VOCABULARIES
'''
tokenizer = BertTokenizer.from_pretrained('bert-large-cased', do_lower_case=False)

In [12]:
'''
BUILDING SENTENCES AS SPACE SEPARATED SEQUENCES OF TOKENS FROM EACH OF OUR GROUPED TOKENS 
Indices of sentences does not match SENTENCE_NUMBER of their tokens.
Perhaps using while x < len(getter.sentences): would work better than 
for sent in getter.sentences, below (to preserve sentence order)
'''
sentences = [' '.join([s[0] for s in sent]) for sent in getter.sentences]
print(sentences[346])
sn = 0
max_sentence_length = 0
while sn < len(sentences):
  print(sn, sentences[sn])

  sn += 1

for sent in getter.sentences:
  print('>>>', sent)

print('Longest sentence:', max_sentence_length)


However , the number of women who can not have the IUS inserted is small .
0 One note ended with the words : ' School is an obstacle which I just do n't have the energy to overcome .
1 Asked for his views on the implications of the case , he said : ' I would ask other publicans to be wary .
2 He called Cameron &quot; feckless , selfish , arrogant &quot; .
3 Mrs Kent , a mother-of-two from Orpington , Kent , can no longer lead a normal life .
4 Feb 23 ; 324 ( 7335 ) : 466 - 9 .
5 As mentioned above , exercise can make symptoms worse for many people with asthma .
6 Use a water based preparation ( rather than an alcohol based one ) .
7 The mediastinum is part of the chest and contains the heart , thymus , oesophagus ( gullet ) , trachea ( windpipe ) , and various nerves and lymph nodes .
8 For example , if a mother has pre-eclampsia , her blood pressure may be treated with medication ,and , depending on her gestation , she may be induced ( to have the baby early ) .
9 But , it is uncommon

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
'''
LISTING THE CLASS LABELS FROM COLUMN 2 IN THE DATAFRAME
'''
labels = [[s[2] for s in sent] for sent in getter.sentences]
print(labels[2615])

'''
Now for sequences of labels, we need to add extra tags to cover multi-segment words split by the BERT tokenizer.
After this procedure, the number of class labels in the sequence should be equal to the number
of segments identified by the BERT tokenizer, with appropriate class labels assigned to each subword
segment.
'''
this_seq=0

new_labels = []
while this_seq < len(sentences):

  sent = sentences[this_seq].split()    
  this_word = 0
  this_seq_new_labels = []

  while this_word < len(sent):
    label_for_this_word = labels[this_seq][this_word]
    segments_this_word = tokenizer.tokenize(sent[this_word])

    z=0
    while z < len(segments_this_word):
      this_seq_new_labels.append(label_for_this_word)
      z += 1

    this_word += 1

  new_labels.append(this_seq_new_labels)

  this_seq += 1


print(sentences[2615]) 
print(tokenizer.tokenize(sentences[2615]))
print(new_labels[2615])

labels = new_labels

['ESMAdvP', 'ESMAdvP', 'ESMAdvP', 'ESMAdvP', 'ESMAdvP', 'ESMAdvP', 'ESMAdvP', 'ESMAdvP', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV']
In another moment down went Alice after it , never once considering how in the world she was to get out again .
['In', 'another', 'moment', 'down', 'went', 'Alice', 'after', 'it', ',', 'never', 'once', 'considering', 'how', 'in', 'the', 'world', 'she', 'was', 'to', 'get', 'out', 'again', '.']
['ESMAdvP', 'ESMAdvP', 'ESMAdvP', 'ESMAdvP', 'ESMAdvP', 'ESMAdvP', 'ESMAdvP', 'ESMAdvP', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV', 'SSMV']


In [14]:
'''
CONVERTING CLASS LABELS TO NUMERICAL VALUES
'''
tags_vals = list(set(data['TAG'].values))

tag2idx = {t: i for i, t in enumerate(tags_vals)}

print('===>', tags_vals) 
print('+++>', tag2idx)

'''
For model CCV.BERT.model.BIO.developmentTrainingAll.0.8231017770597737_20191108.pt:
['B_IN_COMPOUND', 'UNKNOWN', 'ERROR', 'OTHER', 'NOT_CLAUSE_COORDINATOR', 'IN_COMPOUND']
{'B_IN_COMPOUND': 0, 'UNKNOWN': 1, 'ERROR': 2, 'OTHER': 3, 'NOT_CLAUSE_COORDINATOR': 4, 'IN_COMPOUND': 5}

For model
tags_vals = ['CMN1', 'CLAdv', 'CLP', 'SSMAdvP', 'CIN', 'CPA', 'CMP2', 'ESCCV', 'SSCCV', 'ESMV', 'CCV', 'CLV', 'SSMP', 'ESMI', 'SSMA', 'CMAdv', 'STQ', 'COMBINATORY', 'CMN4', 'CMV1', 'SSMN', 'ESMAdvP', 'CLA', 'HELP', 'CMP', 'ESAdvP', 'SPECIAL', 'ESMA', 'CLQ', 'ESMP', 'CMV2', 'ESCM', 'ESMN', 'SSMV', 'CMA1', 'NA', 'CMN2', 'CMV3', 'SSCM', 'CLN', 'SSMI']
tag2idx = {'CMN1': 0, 'CLAdv': 1, 'CLP': 2, 'SSMAdvP': 3, 'CIN': 4, 'CPA': 5, 'CMP2': 6, 'ESCCV': 7, 'SSCCV': 8, 'ESMV': 9, 'CCV': 10, 'CLV': 11, 'SSMP': 12, 'ESMI': 13, 'SSMA': 14, 'CMAdv': 15, 'STQ': 16, 'COMBINATORY': 17, 'CMN4': 18, 'CMV1': 19, 'SSMN': 20, 'ESMAdvP': 21, 'CLA': 22, 'HELP': 23, 'CMP': 24, 'ESAdvP': 25, 'SPECIAL': 26, 'ESMA': 27, 'CLQ': 28, 'ESMP': 29, 'CMV2': 30, 'ESCM': 31, 'ESMN': 32, 'SSMV': 33, 'CMA1': 34, 'NA': 35, 'CMN2': 36, 'CMV3': 37, 'SSCM': 38, 'CLN': 39, 'SSMI': 40}
'''

===> ['SPECIAL', 'ESCCV', 'CMA1', 'SSCCV', 'CIN', 'COMBINATORY', 'SSMV', 'SSCM', 'ESMV', 'CLA', 'HELP', 'CMP', 'ESCM', 'CMN4', 'CMN1', 'CLN', 'ESMAdvP', 'CMV2', 'V', 'ESMN', 'CMP2', 'CCV', 'CMV1', 'CMN2', 'CLP', 'ESAdvP', 'ESMI', 'SSMI', 'ESMP', 'SSMN', 'SSMA', 'CLAdv', 'CLQ', 'CMAdv', 'CPA', 'CMV3', 'SSMAdvP', 'SSMP', 'STQ', 'CMN3', 'CLV', 'NA', 'ESMA']
+++> {'SPECIAL': 0, 'ESCCV': 1, 'CMA1': 2, 'SSCCV': 3, 'CIN': 4, 'COMBINATORY': 5, 'SSMV': 6, 'SSCM': 7, 'ESMV': 8, 'CLA': 9, 'HELP': 10, 'CMP': 11, 'ESCM': 12, 'CMN4': 13, 'CMN1': 14, 'CLN': 15, 'ESMAdvP': 16, 'CMV2': 17, 'V': 18, 'ESMN': 19, 'CMP2': 20, 'CCV': 21, 'CMV1': 22, 'CMN2': 23, 'CLP': 24, 'ESAdvP': 25, 'ESMI': 26, 'SSMI': 27, 'ESMP': 28, 'SSMN': 29, 'SSMA': 30, 'CLAdv': 31, 'CLQ': 32, 'CMAdv': 33, 'CPA': 34, 'CMV3': 35, 'SSMAdvP': 36, 'SSMP': 37, 'STQ': 38, 'CMN3': 39, 'CLV': 40, 'NA': 41, 'ESMA': 42}


"\nFor model CCV.BERT.model.BIO.developmentTrainingAll.0.8231017770597737_20191108.pt:\n['B_IN_COMPOUND', 'UNKNOWN', 'ERROR', 'OTHER', 'NOT_CLAUSE_COORDINATOR', 'IN_COMPOUND']\n{'B_IN_COMPOUND': 0, 'UNKNOWN': 1, 'ERROR': 2, 'OTHER': 3, 'NOT_CLAUSE_COORDINATOR': 4, 'IN_COMPOUND': 5}\n\nFor model\ntags_vals = ['CMN1', 'CLAdv', 'CLP', 'SSMAdvP', 'CIN', 'CPA', 'CMP2', 'ESCCV', 'SSCCV', 'ESMV', 'CCV', 'CLV', 'SSMP', 'ESMI', 'SSMA', 'CMAdv', 'STQ', 'COMBINATORY', 'CMN4', 'CMV1', 'SSMN', 'ESMAdvP', 'CLA', 'HELP', 'CMP', 'ESAdvP', 'SPECIAL', 'ESMA', 'CLQ', 'ESMP', 'CMV2', 'ESCM', 'ESMN', 'SSMV', 'CMA1', 'NA', 'CMN2', 'CMV3', 'SSCM', 'CLN', 'SSMI']\ntag2idx = {'CMN1': 0, 'CLAdv': 1, 'CLP': 2, 'SSMAdvP': 3, 'CIN': 4, 'CPA': 5, 'CMP2': 6, 'ESCCV': 7, 'SSCCV': 8, 'ESMV': 9, 'CCV': 10, 'CLV': 11, 'SSMP': 12, 'ESMI': 13, 'SSMA': 14, 'CMAdv': 15, 'STQ': 16, 'COMBINATORY': 17, 'CMN4': 18, 'CMV1': 19, 'SSMN': 20, 'ESMAdvP': 21, 'CLA': 22, 'HELP': 23, 'CMP': 24, 'ESAdvP': 25, 'SPECIAL': 26, 'ESMA': 27, 

In [15]:
'''
SETTING SEQUENCE LENGTH AND BATCH SIZE
'''
MAX_LEN = 200
bs = 6
# bs = 800
# bs = 8 # batch size (bs) is a good way to manage the amount of memory used by the model

In [16]:
'''
USE A GPU IF ONE IS AVAILABLE, OTHERWISE USE A CPU
'''
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_gpu = torch.cuda.device_count()

In [17]:
'''
DETERMINING THE NAME OF THE GPU WE ARE USING VIA GOOGLE'S CLOUD SERVICES
'''
torch.cuda.get_device_name(0)

'GeForce RTX 2080 SUPER'

In [18]:
'''
CUTTING AND PADDING LABEL SEQUENCES TO MATCH DESIRED LENGTH
'''

tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels], 
                     maxlen=MAX_LEN, value=tag2idx['NA'], padding='post', 
                     dtype='long', truncating='post')
print(tags.shape)

(15352, 200)


In [19]:
'''
TOKENIZING ALL SENTENCES
'''
print(len(sentences))
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts[2000])
print(tags[2000])



15352
['Had', 'the', 'changes', 'of', 'a', 'lifetime', 'been', 'crowded', 'into', 'so', 'brief', 'a', 'space', ',', 'and', 'were', 'they', 'now', 'four', 'aged', 'people', ',', 'sitting', 'with', 'their', 'old', 'friend', ',', 'Doctor', 'He', '##ide', '##gger', '?']
[ 1  1  1  1  1  1  1  1  1  1  1  1  1 21 21 21 21 21 21 21 21  6  6  6
  6  6  6 29 29 29 29 29 29 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41]


In [20]:
'''
CUTTING AND PADDING TOKEN SEQUENCES TO MATCH DESIRED LENGTH

Note here that shape of input_ids will differ for each test file.
Health needs 1001 test sequences
Literature needs 625 test sequences
News needs 1002 test sequences
'''
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts], maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
print(input_ids.shape)

(15352, 200)


In [21]:
'''
CREATING AN ATTENTION MASK SO THAT BERT IGNORES THE PADDED ELEMENTS OF THE SEQUENCES
'''
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
print(input_ids.shape, len(attention_masks))

(15352, 200) 15352


In [22]:
'''
CONSIDER DELETING: THIS CELL MAY BE IRRELEVANT FOR THE CURRENT TRAINING TASK


CREATING THE TEST DATA BY IDENTIFYING RELEVANT SLICES OF LISTS: input_ids, tags,
and attention masks 

Eventually, the test data will be derived from a single sequence input as a 
csv_file, tagged with the id for 'UNKNOWN' via tag2idx['UNKNOWN']
Remember that sequences are now sorted: HEALTH_TESTING, LITERATURE_TESTING, 
NEWS_TESTING, TRAINING
Note: this order is random on every run. See comment below.

Keep in mind that sentences are ordered according to the procedure in cell 55
(Ordered by the ['TYPE'] column created in cell 54)
'''

# N.B. THESE  SENTENCE INDEXES CHANGE ON EVERY RUN BECAUSE sentence getter group order is 
# random on each run
tr_inputs = input_ids[:2165] # Daily Mail data starts at 5309
tr_tags = tags[:2165]
tr_attention_masks = attention_masks[:2165]

# tr_inputs = input_ids[2634:]
# tr_tags = tags[2634:]
# tr_attention_masks = attention_masks[2634:]


# te_inputs = input_ids[1631:2633]
# te_tags = tags[1631:2633]
# te_attention_masks = attention_masks[1631:2633]
'''
Changing this so that we test on ALL testing files together
'''
te_inputs = input_ids[2166:3168] #HEALTH
te_tags = tags[2166:3168]
te_attention_masks = attention_masks[2166:3168]

te_inputs = input_ids[2166:3168]
te_tags = tags[2166:3168]
te_attention_masks = attention_masks[2166:3168]

'''
Record results from each run:
CCV Testing
Register F1
Health 
Literature
News
'''

print('3-->', te_inputs.shape)

3--> (1002, 200)


In [23]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [24]:
'''
Converting the dataset to torch tensors for use with pytorch
'''
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [25]:
'''
DEFINING DATALOADERS
'''
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [26]:
'''
SETTING UP THE BERT MODEL FOR FINETUNING AND PROVIDING THE NUMBER OF CLASS LABELS
'''
model = BertForTokenClassification.from_pretrained('bert-large-cased', num_labels=len(tag2idx))
model_type = 'TokenClassification'
# model = BertForSequenceClassification.from_pretrained('bert-large-cased', num_labels=len(tag2idx))
# model_type = 'SequenceClassification'

100%|██████████| 1242874899/1242874899 [25:06<00:00, 824787.62B/s] 


In [27]:
'''
PASSING THE MODEL PARAMETERS TO THE GPU
'''
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
          

In [31]:
'''
SETTING UP THE OPTIMIZER AND ADDING THE PARAMETERS THAT IT SHOULD UPDATE
Specifying the Adam optmiizer
and adding some weight decay as regularization to the main weight matrices
'''
FULL_FINETUNING = True
if FULL_FINETUNING:
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'gamma', 'beta']
  optimizer_grouped_parameters = [
                                  {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                                   'weight_decay_weight': 0.01},
                                  {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                                   'weight_decay_rate': 0.0}
  ]
else:
  param_optimizer = list(model.classifier.named_parameters())
  optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [32]:
pip install seqeval[gpu]

In [33]:
'''
DEFINING EVALUATION METRIC FOR USE IN FINETUNING (F1)
'''
from seqeval.metrics import f1_score

'''
Here, I think we need to modify flat_accuracy so that non-sign tokens are deleted from both pred_flat and labels_flat
'''
def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis=2).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [34]:
'''
New temp cell for testing updated flat_accuracy

Idea is only to evaluate classification of signs of syntactic complexity, not all tokens
Question is how to access these_words from this function. Is it coming from 
token_type_ids 
or 
b_input_ids 
? Need to share indices with labels and preds
'''

these_words = ['xxx', 'yyy', 'zzz', 'that', 'aaa', 'bbb', 'and', 'ccc', 'ddd']
labels = ['NA', 'NA', 'NA', 'SSCCV', 'SSCCV', 'SSCCV', 'CMN1', 'CMN1', 'CMN1']
preds = ['NA', 'NA', 'NA', 'SSCCV', 'SSCCV', 'SSCCV', 'CMN1', 'CMN1', 'CMN1']

# want eval_words = ['that', 'and']
# want eval_labels = ['SSCCV', 'CMN1']
# wamt eval_preds = ['SSCCV', 'CMN1']

In [40]:
'''
FINETUNING

Copy pasted from tutorial
'''
epochs = 5
max_grad_norm = 1.0

print('Started finetuning at', str(datetime.datetime.now()))
for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        

        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        


        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]

    '''
    Here could be the point to create and edit temp_pred_tags (from pred_tags) and temp_valid_tags from valid_tags to delete offsets from these lists 
    for which temp_valid_tags[offset] is 'NA' 
    '''
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

    print('Finished finetuning epoch at', str(datetime.datetime.now()))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Started finetuning at 2020-07-14 15:32:51.922858


RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.cuda.IntTensor instead (while checking arguments for embedding)